In [4]:
import src.config as c
import pandas as pd
import matplotlib.pyplot as plt
import json
import sqlite3

plt.rcParams.update({"text.usetex": True, "font.family": "monospace", "font.size": "14"})

## RQ 2.2: Query Size Success


,exact_matches,inexact_matches,inexact_matches_correct_line,non_matches,avg_query_size,runtime,line_of_interest
10,0,10,0,0,3327,107,5
20,0,10,0,0,6567,128,14
30,0,10,0,0,9180,132,11
40,0,0,0,10,11394,131,19
50,0,10,0,0,13445,89,43
60,0,0,0,10,15415,113,3
70,0,10,0,0,17874,108,50
80,0,0,0,10,20088,111,33
90,0,0,0,10,22476,113,48
100,0,0,0,10,24736,130,53


In [4]:
selected = data['exact_matches'] + data['inexact_matches'] + data['inexact_matches_correct_line']
print(selected.to_latex())

\begin{tabular}{lr}
\toprule
 & 0 \\
\midrule
10 & 10 \\
20 & 10 \\
30 & 10 \\
40 & 0 \\
50 & 10 \\
60 & 10 \\
70 & 10 \\
80 & 10 \\
90 & 0 \\
100 & 0 \\
110 & 0 \\
120 & 0 \\
130 & 0 \\
140 & 10 \\
150 & 0 \\
160 & 0 \\
170 & 0 \\
180 & 0 \\
190 & 10 \\
200 & 0 \\
210 & 0 \\
220 & 0 \\
230 & 0 \\
240 & 0 \\
250 & 0 \\
260 & 0 \\
270 & 0 \\
280 & 0 \\
290 & 0 \\
300 & 0 \\
310 & 0 \\
320 & 0 \\
330 & 0 \\
340 & 0 \\
350 & 0 \\
360 & 0 \\
370 & 0 \\
380 & 0 \\
390 & 0 \\
400 & 0 \\
410 & 10 \\
420 & 0 \\
430 & 0 \\
440 & 0 \\
450 & 0 \\
460 & 0 \\
470 & 0 \\
480 & 0 \\
490 & 0 \\
500 & 10 \\
\bottomrule
\end{tabular}

